Tech Challenge


* Créditos da base de dados:
https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning



In [ ]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install matplotlib


# from google.colab import drive
# drive.mount('/content/drive/')

# /content/drive/MyDrive/insurance.csv



Mounted at /content/drive/


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# para deixar todas as saídas com os mesmos valores obtidos na live.
import numpy as np
np.random.seed(42)

# Importando a base de dados
dados_original = pd.read_csv("../Base_de_dados/insurance.csv", sep=",")

# Créditos para a base de dados importada do projeto: https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning
# dados_original = pd.read_csv("../Base_de_dados/insurance_dataset.csv", sep=",")
# Retirando as colunas não utilizadas no Tech Challenge
# dados_original.drop(['medical_history'], axis=1, inplace=True)
# dados_original.drop(['family_medical_history'], axis=1, inplace=True)
# dados_original.drop(['exercise_frequency'], axis=1, inplace=True)
# dados_original.drop(['occupation'], axis=1, inplace=True)
# dados_original.drop(['coverage_level'], axis=1, inplace=True)

dados_original.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [2]:
# Informações estruturais sobre o dataframe
dados_original.shape



(1338, 7)

In [3]:
# Estatísticas sobre o dataframe
display(dados_original.describe())

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [4]:
# Verificando a ausência de informações
dados_original.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [6]:
print("Informações sobre o DataFrame dados:")
print(dados_original.info())

Informações sobre o DataFrame dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None


In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

dados=dados_original.copy()

# Ajustar e transformar os rótulos
dados['sex'] = label_encoder.fit_transform(dados['sex'])
dados['smoker'] = label_encoder.fit_transform(dados['smoker'])
dados['region'] = label_encoder.fit_transform(dados['region'])

dados.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,2,1725.55230
2,28,1,33.000,3,0,2,4449.46200
3,33,1,22.705,0,0,1,21984.47061
4,32,1,28.880,0,0,1,3866.85520


In [8]:
from sklearn.preprocessing import StandardScaler

features = ['age', 'sex', 'bmi', 'children', 'smoker', 'region'] #, 'charges']
X = dados[features].values
y = dados['charges'].values

In [ ]:
X = StandardScaler().fit_transform(X)
df_padronizado = pd.DataFrame(data=X, columns=features)
display(df_padronizado.head())

,age,sex,bmi,children,smoker,region
0,-1.438764,-1.010519,-0.453320,-0.908614,1.970587,1.343905
1,-1.509965,0.989591,0.509621,-0.078767,-0.507463,0.438495
2,-0.797954,0.989591,0.383307,1.580926,-0.507463,0.438495
3,-0.441948,0.989591,-1.305531,-0.908614,-0.507463,-0.466915
4,-0.513149,0.989591,-0.292556,-0.908614,-0.507463,-0.466915


Testando regressão linear múltipla

In [9]:
! pip install statsmodels

  Obtaining dependency information for statsmodels from https://files.pythonhosted.org/packages/7f/0a/28b3b3c807a518b7d0ed0ae45bfdd6be23b38a0c60790f90ce1f8383a640/statsmodels-0.14.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for patsy>=0.5.6 from https://files.pythonhosted.org/packages/43/f3/1d311a09c34f14f5973bb0bb0dc3a6e007e1eda90b5492d082689936ca51/patsy-0.5.6-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 14.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 21.1 MB/s eta 0:00:00


In [10]:
import statsmodels.api as sm

# Definir as variáveis independentes (X)
X = dados[['age', 'sex', 'bmi', 'children', 'smoker', 'region']]
# Definir a variável dependente (Y)
Y = dados['charges']

# Criar e ajustar o modelo de regressão linear múltipla
modelo = sm.OLS(Y, X).fit()

# Imprimir os resultados do modelo
print(modelo.summary())

                                 OLS Regression Results                                
Dep. Variable:                charges   R-squared (uncentered):                   0.874
Model:                            OLS   Adj. R-squared (uncentered):              0.873
Method:                 Least Squares   F-statistic:                              1537.
Date:                Sat, 04 May 2024   Prob (F-statistic):                        0.00
Time:                        15:38:13   Log-Likelihood:                         -13621.
No. Observations:                1338   AIC:                                  2.725e+04
Df Residuals:                    1332   BIC:                                  2.729e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [11]:
# ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
print(modelo.predict([41, 1, 27.755, 0, 0, 1]))
print(modelo.predict([41, 1, 18.5, 0, 0, 1]))


[8663.2916433]
[8086.61231029]


In [ ]:
! pip install scikit-learn
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# realizar as previsões
y_pred = modelo.predict(X)

# Calcular MAE, MSE e RMSE
mae = mean_absolute_error(Y, y_pred)
mse = mean_squared_error(Y, y_pred)
rmse = np.sqrt(mse)

# Imprimir as métricas
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

MAE: 4399.280332807504
MSE: 40727376.333000176
RMSE: 6381.800399025355


In [ ]:
from sklearn.model_selection import train_test_split

# dados_rlm = dados.copy()
dados_rlm = df_padronizado.copy()

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)

# Calculando o R2
r2 = model.score(X_test, y_test)

# Calculando o MSE
mse = mean_squared_error(y_test, y_predict)

import math

# Calculando o RMSE
rmse = math.sqrt(mse)

# Imprimindo os resultados
print("R²:", r2)
print("MSE:", mse)
print("RMSE:", rmse)



R²: 0.7833463107364536
MSE: 33635210.43117845
RMSE: 5799.5870914383595


In [ ]:
# Acessando os coeficientes do modelo
print("Coeficientes:", model.coef_)

Coeficientes: [ 3.61028043e+03 -9.39521400e+00  2.04689296e+03  5.12253132e+02
  9.54291505e+03 -2.99625864e+02]


In [ ]:
# ['age', 'sex', 'bmi', 'children', 'smoker', 'region']
print(model.predict([[41, 1, 27.755, 0, 0, 1]]))
print(model.predict([[41, 1, 18.5, 0, 0, 1]]))
# array([198895.44142703])

[217839.43574274]
[198895.44142703]
